In [ ]:
!pip install --upgrade git+https://github.com/huggingface/transformers.git byaldi accelerate qwen_vl_utils pdf2image
!sudo apt-get install -y poppler-utils

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-b95jcdf8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-b95jcdf8
  Resolved https://github.com/huggingface/transformers.git to commit c8c8dffbe45ebef0a8dba4a51024e5e5e498596b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Byaldi-0.0.7-py3-none-any.whl.metadata (20 kB)
  Using cached colpali_engine-0.3.4-py3-none-any.whl.metadata (21 kB)
INFO: pip is looking at multiple versions of colpali-engine to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.0/260.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling h

In [ ]:
from byaldi import RAGMultiModalModel
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
from pdf2image import convert_from_path
import base64
import requests
from urllib.parse import urlencode
import zipfile
import io
import shutil
import os
from mistralai import Mistral

# Загрузка датасета

In [ ]:
def load_data(url: str):
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    final_url = base_url + urlencode(dict(public_key=url))
    response = requests.get(final_url)

    download_url = response.json()['href']
    download_response = requests.get(download_url)

    zip = zipfile.ZipFile(io.BytesIO(download_response.content))
    zip.extractall('/content/')

In [ ]:
load_data("https://disk.yandex.ru/d/_Y0wCNLTFevIBw")

In [ ]:
with zipfile.ZipFile('/content/3 задача/RAG.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [ ]:
listdir = os.listdir()

if 'pdf_files' in listdir:
    shutil.rmtree('pdf_files')

if 'docx_files' in listdir:
    shutil.rmtree('docx_files')

os.mkdir('pdf_files')
os.mkdir('docx_files')

listdir = os.listdir()

for filename in listdir:
    if filename.split('.')[-1] == 'pdf':
        os.rename(filename, 'pdf_files/' + filename)

    elif filename.split('.')[-1] == 'docx':
        os.rename(filename, 'docx_files/' + filename)

# Создание базы знаний

In [ ]:
RAG = RAGMultiModalModel.from_pretrained("vidore/colqwen2-v1.0", index_root='/content/indices')

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
RAG.index(
    input_path="/content/pdf_files/", # The path to your documents
    index_name='testindex1', # The name you want to give to your index. It'll be saved at `index_root/index_name/`.
    store_collection_with_index=False, # Whether the index should store the base64 encoded documents.
    overwrite=True # Whether to overwrite an index if it already exists. If False, it'll return None and do nothing if `index_root/index_name` exists.
)

Indexing file: /content/pdf_files/sr_ru_annual_report_pages_nornik_2022.pdf
Added page 1 of document 0 to index.
Added page 2 of document 0 to index.
Added page 3 of document 0 to index.
Added page 4 of document 0 to index.
Added page 5 of document 0 to index.
Added page 6 of document 0 to index.
Added page 7 of document 0 to index.
Added page 8 of document 0 to index.
Added page 9 of document 0 to index.
Added page 10 of document 0 to index.
Added page 11 of document 0 to index.
Added page 12 of document 0 to index.
Added page 13 of document 0 to index.
Added page 14 of document 0 to index.
Added page 15 of document 0 to index.
Added page 16 of document 0 to index.
Added page 17 of document 0 to index.
Added page 18 of document 0 to index.
Added page 19 of document 0 to index.
Added page 20 of document 0 to index.
Added page 21 of document 0 to index.
Added page 22 of document 0 to index.
Added page 23 of document 0 to index.
Added page 24 of document 0 to index.
Added page 25 of docu

{0: '/content/pdf_files/sr_ru_annual_report_pages_nornik_2022.pdf',
 1: '/content/pdf_files/responsible_supply_chain_report_rus.pdf',
 2: '/content/pdf_files/NN_CSO2021_RUS_03.03.2023.pdf',
 3: '/content/pdf_files/2_5366183129474161642.pdf',
 4: '/content/pdf_files/digital_production_5.pdf',
 5: '/content/pdf_files/2022_Annual_Report_of_PJSC_MMC_Norilsk_Nickel_rus.pdf',
 6: '/content/pdf_files/McKinsey_Next Big Arenas_2024 (213 pgs).pdf',
 7: '/content/pdf_files/NN_AR_2021_Book_RUS_26.09.22.pdf',
 8: '/content/pdf_files/îîè 2024.pdf',
 9: '/content/pdf_files/ìïîè 2024.pdf',
 10: '/content/pdf_files/KPMG_Global Metals & Mining_2024 (48 pgs).pdf',
 11: '/content/pdf_files/Kept_Äíº«α_µÑ¡_ó_¼ÑΓá½½παú¿τÑß¬«⌐_«Γαáß½¿_2è2024.pdf',
 12: '/content/pdf_files/É«ß¬«¡úαÑßß_Éδ¡«¬_»α«¼δΦ½Ñ¡¡δσ_α«í«Γ«ó_ó_¼¿αÑ_¿_É«ßß¿¿_2024_16_ßΓα.pdf',
 13: '/content/pdf_files/ä«¬½áñ, πú«½∞ τáßΓ∞ 1.pdf',
 14: '/content/pdf_files/û¿Σα«ó¿ºáµ¿∩_ú«α¡«_¼ÑΓá½½παú¿τÑß¬«⌐_«Γαáß½¿_É«ßß¿¿_ó_2024_ú_.pdf',
 15: '/content/pdf_file

In [ ]:
results = RAG.search('Какой прогноз роста производства продукции компании Норникель к 2030?', k=5)

In [ ]:
results

[{'doc_id': 24, 'page_num': 10, 'score': 30.25, 'metadata': {}, 'base64': None},
 {'doc_id': 0, 'page_num': 7, 'score': 29.875, 'metadata': {}, 'base64': None},
 {'doc_id': 20, 'page_num': 1, 'score': 29.75, 'metadata': {}, 'base64': None},
 {'doc_id': 7, 'page_num': 22, 'score': 29.75, 'metadata': {}, 'base64': None},
 {'doc_id': 23, 'page_num': 3, 'score': 29.625, 'metadata': {}, 'base64': None}]

In [ ]:
def zip_folder(folder_path, output_zip_path):
    with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Полный путь к файлу
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

folder_to_zip = '/content/indices/testindex1'
output_zip_file = 'rag.zip'  # Имя выходного ZIP-файла

zip_folder(folder_to_zip, output_zip_file)
print(f"Папка '{folder_to_zip}' успешно заархивирована в '{output_zip_file}'")

Папка '/content/indices/testindex1' успешно заархивирована в 'rag.zip'
